In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../Data"))

['sample_submission.csv', 'test_identity.csv', 'test_transaction.csv', 'train_identity.csv', 'train_transaction.csv']


In [2]:
from sklearn import preprocessing
import xgboost as xgb

In [5]:
%%time
train_transaction = pd.read_csv('../Data/train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('../Data/test_transaction.csv', index_col='TransactionID')

train_identity = pd.read_csv('../Data/train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('../Data/test_identity.csv', index_col='TransactionID')

sample_submission = pd.read_csv('../Data/sample_submission.csv', index_col='TransactionID')

train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

print(train.shape)
print(test.shape)

y_train = train['isFraud'].copy()
del train_transaction, train_identity, test_transaction, test_identity

# Drop target, fill in NaNs
X_train = train.drop('isFraud', axis=1)
X_test = test.copy()

del train, test

X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)

# Label Encoding
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))   

(590540, 433)
(506691, 432)
Wall time: 3min 10s


In [14]:
clf = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=9,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    missing=-999,
    random_state=2019,
    tree_method='hist'
)


In [15]:
%time clf.fit(X_train, y_train)

[11:14:17] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
Wall time: 13min 7s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=9, min_child_weight=1, missing=-999, n_estimators=500,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=2019, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.9, tree_method='hist')

In [16]:
sample_submission['isFraud'] = clf.predict_proba(X_test)[:,1]
sample_submission.to_csv('simple_xgboost.csv')

In [20]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(X_test, sample_submission)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

ValueError: Classification metrics can't handle a mix of continuous-multioutput and continuous targets